# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
#Your code here
import pandas as pd

# Load the dataset
data = pd.read_csv('homepage_actions.csv')

# Explore the dataset
print(data.head())  # View the first few rows
print(data.info())  # Get column information and data types
print(data.describe())  # Summary statistics

# Investigate the 'id' column
total_viewers = data['id'].nunique()  # Total number of viewers
clickers = data[data['action'] == 'click']['id'].nunique()  # Number of viewers who clicked

print("Total Viewers:", total_viewers)
print("Clickers:", clickers)

# Check for anomalies
anomalies = data[(data['action'] == 'click') & ~(data['id'].isin(data[data['action'] == 'view']['id']))]
if anomalies.empty:
    print("No anomalies found.")
else:
    print("Anomalies found.")

# Identify overlap between control and experiment groups
control_group = data[data['group'] == 'control']['id']
experiment_group = data[data['group'] == 'experiment']['id']
overlap = control_group[control_group.isin(experiment_group)]
if overlap.empty:
    print("No overlap between control and experiment groups.")
else:
    print("Overlap between control and experiment groups.")

# Account for overlap in experimental design
# You can exclude the overlapping IDs from analysis or assign them randomly to either group

                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None
                  id
count    8188.000000
mean   564699.749878
std    219085.845672
min    182988.000000
25%    373637.500000
50%    566840.500000
75%    758078.000000
max    937217.000000
Total Viewers: 6328
Clickers: 1860
No anomalies found.
No overlap betwee

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [3]:
#Your code here
import pandas as pd
from scipy.stats import chi2_contingency

# Load the dataset
data = pd.read_csv('homepage_actions.csv')

# Create a contingency table of observed frequencies
contingency_table = pd.crosstab(data['group'], data['action'])

# Perform chi-square test
chi2, p_value, _, _ = chi2_contingency(contingency_table)

# Print the results
print("Chi-square statistic:", chi2)
print("P-value:", p_value)

# Interpret the results
alpha = 0.05  # significance level
if p_value < alpha:
    print("The experimental homepage was significantly more effective than the control group.")
else:
    print("There is no significant difference between the experimental homepage and the control group.")

Chi-square statistic: 3.636160051233291
P-value: 0.056537191086915774
There is no significant difference between the experimental homepage and the control group.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [5]:
#Your code here
control_group = data[data['group'] == 'control']
control_ctr = control_group[control_group['action'] == 'click'].shape[0] / control_group.shape[0]

experiment_group = data[data['group'] == 'experiment']
expected_clicks = control_ctr * experiment_group.shape[0]

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [6]:
#Your code here
import math

# Observed clicks in the experiment group
observed_clicks = experiment_group[experiment_group['action'] == 'click'].shape[0]

# Calculate the standard deviation
standard_deviation = math.sqrt(experiment_group.shape[0] * control_ctr * (1 - control_ctr))

# Calculate the number of standard deviations
z_score = (observed_clicks - expected_clicks) / standard_deviation

print("Number of standard deviations:", z_score)

Number of standard deviations: 2.71607302278784


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [7]:
#Your code here

from scipy.stats import norm

# Calculate the p-value
p_value = 1 - norm.cdf(z_score)

print("P-value:", p_value)

P-value: 0.0033030672759265522


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**
In the previous analysis using the chi-square test, the obtained p-value was 0.0565, indicating that there was no significant difference between the experimental homepage and the control group.

However, in the current analysis using the z-score calculation, the obtained p-value is 0.0033, which is below the typical significance level of 0.05. This suggests that there is a significant difference between the experimental homepage and the control group.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.